In [2]:
!pip install synthcity
!pip install opacus==1.4.0


In [3]:
# !unrar x data.rar

In [4]:
import pandas as pd
import numpy as np
import torch
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from hyperopt import fmin, rand, hp, Trials, STATUS_OK
from synthcity.plugins import Plugins
from synthcity.plugins.generic.plugin_ddpm import TabDDPMPlugin as DDPM
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import tqdm

E0000 00:00:1749405272.599162      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749405272.645771      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [5]:
dir_datasets = '/kaggle/input/nir02-main/data/'

# Загрузка реальных датасетов
real_data_1 = pd.read_csv(dir_datasets+'insurance.csv')
real_data_2 = pd.read_csv(dir_datasets+'my_classification.csv')
real_data_3 = pd.read_csv(dir_datasets+'my_regression.csv')
real_data_4 = pd.read_csv(dir_datasets+'two_moons.csv')
real_data_5 = pd.read_csv(dir_datasets+'california_housing.csv')
real_data_6 = pd.read_csv(dir_datasets+'titanic.csv')

datasets = {
        'insurance': {
                        "data": real_data_1,
                        "task": "regression",
                        "target": "expenses",
                        "num_columns":
                        ["age", "bmi", "children"],
                        "cat_columns":
                        ["sex", "smoker", "region"]
                    },
        'my_classification': {
                        "data": real_data_2,
                        "task": "classification",
                        "target": "target",
                        "num_columns":
                        ["feature_0", "feature_1", "feature_2", "feature_3", "feature_4"],
                        "cat_columns":
                        []
                    },
        'my_regression': {
                        "data": real_data_3,
                        "task": "regression",
                        "target": "target",
                        "num_columns":
                        ["feature_0", "feature_1", "feature_2", "feature_3"],
                        "cat_columns":
                        []
                    },

        'two_moons': {
                        "data": real_data_4,
                        "task": "classification",
                        "target": "label",
                        "num_columns":
                        ["x1", "x2"],
                        "cat_columns": 
                        []
                    },
        
    }

for name, data in datasets.items():
    print(f" Информация о датасете {name}:")
    print(f" Количество строк: {data['data'].shape[0]}")
    print(f" Количество колонок: {data['data'].shape[1]}")
    print(f" Колонки: {list(data['data'].columns)}")
    print(f" Задача: {data['task']}")
    print(f" Целевая переменная: {data['target']}")
    print(f" Числовые признаки: {data['num_columns']}")
    print(f" Категориальные признаки: {data['cat_columns']}")
    print()

 Информация о датасете insurance:
 Количество строк: 1338
 Количество колонок: 7
 Колонки: ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses']
 Задача: regression
 Целевая переменная: expenses
 Числовые признаки: ['age', 'bmi', 'children']
 Категориальные признаки: ['sex', 'smoker', 'region']

 Информация о датасете my_classification:
 Количество строк: 900
 Количество колонок: 6
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'target']
 Задача: classification
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']
 Категориальные признаки: []

 Информация о датасете my_regression:
 Количество строк: 900
 Количество колонок: 5
 Колонки: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'target']
 Задача: regression
 Целевая переменная: target
 Числовые признаки: ['feature_0', 'feature_1', 'feature_2', 'feature_3']
 Категориальные признаки: []

 Информация о датасете two_moons:
 Количе

In [6]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [7]:
def process_data(dataset, num_columns, cat_columns, transformation_num_type='None', transformation_cat_type='None'):

    df_processed = dataset.copy()

    # Обработка числовых признаков
    if transformation_num_type == 'CDF':
        # CDF трансформация: преобразует значения в их эмпирическую функцию распределения
        for col in num_columns:
            # Правильная формула для эмпирической CDF
            df_processed[col] = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

    elif transformation_num_type == 'PLE_CDF':
        # PLE_CDF (Probability Logit Envelope CDF) - более сложная трансформация
        for col in num_columns:
            # Сначала применяем CDF
            cdf_values = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

            # Затем применяем logit трансформацию с небольшим сглаживанием
            # Избегаем 0 и 1 для предотвращения бесконечности в logit
            epsilon = 1e-6
            cdf_values = np.clip(cdf_values, epsilon, 1 - epsilon)

            # Логит трансформация: ln(p/(1-p))
            df_processed[col] = np.log(cdf_values / (1 - cdf_values))

    # Обработка категориальных признаков
    if transformation_cat_type == 'OHE':
        # One Hot Encoding для категориальных признаков
        for col in cat_columns:
            # Создаем dummy переменные с префиксом имени колонки
            dummy_df = pd.get_dummies(df_processed[col], prefix=col, dtype=int)

            # Удаляем исходную категориальную колонку
            df_processed = df_processed.drop(columns=[col])

            # Добавляем новые dummy колонки
            df_processed = pd.concat([df_processed, dummy_df], axis=1)

    return df_processed

In [8]:
# Определение пространства поиска гиперпараметров для DDPM
ddpm_space = {
    'batch_size': hp.choice('batch_size', [4096]),
    'lr': hp.qloguniform('lr', np.log(3.5e-4), np.log(9.2e-4), 1e-5),
    'num_timesteps': hp.choice('num_timesteps', [1000]),
    'n_layers_hidden': hp.choice('n_layers_hidden', [2, 4, 6]),
    'n_units_hidden': hp.choice('n_units_hidden', [256, 512, 1024]),
    'transformation_num_type': hp.choice('transformation_num_type', ['None']),
    'transformation_cat_type': hp.choice('transformation_cat_type', ['None'])
}

In [9]:
def evaluate_c2st(real, synthetic):
    real_copy = real.copy()
    if hasattr(synthetic, "dataframe"):
        synthetic = synthetic.dataframe()

    synthetic_copy = synthetic.copy()

    # Добавление меток
    real_copy['label'] = 1
    synthetic_copy['label'] = 0

    # Объединение данных
    df = pd.concat([real_copy, synthetic_copy], ignore_index=True)

    # Определение категориальных колонок
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
    categorical_columns = [col for col in categorical_columns if col != 'label']

    # Label Encoding для категориальных признаков
    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))

    # Разделение на признаки и метки
    X = df.drop(columns='label')
    y = df['label']

    # Разделение на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=RANDOM_SEED
    )
    # Обучение XGBoost
    clf = XGBClassifier(
        random_state=RANDOM_SEED
    )
    clf.fit(X_train, y_train)

    # Предсказания
    y_pred_proba = clf.predict_proba(X_test)[:, 1]

    return roc_auc_score(y_test, y_pred_proba)


In [10]:
import sys
import os
import builtins
import contextlib

@contextlib.contextmanager
def suppress_all_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        old_print = builtins.print
        builtins.print = lambda *args, **kwargs: None
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr
            builtins.print = old_print

In [11]:
from sklearn.model_selection import KFold

def generate_and_evaluate(params, dataset_info, n_splits=3):
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    columns_dataset = dataset_info['num_columns'] + dataset_info['cat_columns'] + [dataset_info['target']]
    data = process_data(dataset_info['data'][columns_dataset], dataset_info['num_columns'], dataset_info['cat_columns'],
                        params['transformation_num_type'], params['transformation_cat_type'])

    kf = KFold(n_splits=n_splits, shuffle=False)
    c2st_scores = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index].reset_index(drop=True)
        test_data = data.iloc[test_index].reset_index(drop=True)

        # Создание и обучение DDPM с заданными параметрами
        ddpm = DDPM(
            lr=params['lr'],
            num_timesteps=params['num_timesteps'],
            model_params = dict(n_layers_hidden=params['n_layers_hidden'],
                                n_units_hidden=params['n_units_hidden'],
                                dropout=0.0),
            batch_size=params['batch_size'],
            #n_iter=params['n_iter']
        )
        # Обучение модели на реальных данных
        with suppress_all_output():
          ddpm.fit(train_data)

        # Генерация синтетических данных
        synthetic_data = ddpm.generate(len(test_data))

        score = evaluate_c2st(test_data, synthetic_data)
        c2st_scores.append(score)

    return np.mean(c2st_scores)

In [12]:
from hyperopt import space_eval

def optimize_dataset(dataset_name, dataset, max_evals=30):
    print(f"Оптимизация для датасета: {dataset_name}")
    trials = Trials()

    def objective(params):
        print("="*50)
        c2st_score = generate_and_evaluate(params, dataset)
        print(c2st_score)
        print(params)
        print("="*50)
        return {
            'loss': c2st_score,
            'status': STATUS_OK,
            'c2st_score': c2st_score
        }

    rng = np.random.default_rng(RANDOM_SEED)
    best = fmin(
        fn=objective,
        space=ddpm_space,
        algo=rand.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=rng
    )

    # Автоматическое декодирование параметров
    best_params = space_eval(ddpm_space, best)

    # Находим лучший результат
    best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
    best_c2st = best_trial['result'].get('c2st_score', None)

    results = {
        'best_params': best_params,
        'best_c2st': best_c2st,
        'c2st_diff_from_optimal': best_trial['result']['loss']
    }

    print(f"Лучший C2ST для {dataset_name}: {best_c2st}")
    print(f"Лучшие параметры: {best_params}")
    print('-' * 50)

    return results

In [15]:
# Словарь для хранения результатов
all_results = {}

# Указываем количество итераций для каждого датасета
max_evals = 50

# Запускаем оптимизацию для каждого датасета
for dataset_name in datasets:
    data = datasets[dataset_name]
    all_results[dataset_name] = optimize_dataset(dataset_name, data, max_evals)

Оптимизация для датасета: insurance
0.7263310314101136                                    
{'batch_size': 4096, 'lr': 0.00064, 'n_layers_hidden': 6, 'n_units_hidden': 256, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
0.990495284770179                                                                 
{'batch_size': 4096, 'lr': 0.00063, 'n_layers_hidden': 2, 'n_units_hidden': 1024, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
0.9817888171084874                                                                 
{'batch_size': 4096, 'lr': 0.00038, 'n_layers_hidden': 2, 'n_units_hidden': 256, 'num_timesteps': 1000, 'transformation_cat_type': 'None', 'transformation_num_type': 'None'}
0.8415849855201604                                                                 
{'batch_size': 4096, 'lr': 0.00043000000000000004, 'n_layers_hidden': 4, 'n_units_hidden': 512, 'num_timesteps': 1000, 'transformation_

In [16]:
for dataset_name in datasets:
    print(dataset_name, all_results[dataset_name]['best_c2st'])

insurance 0.6649773520457415
my_classification 0.5906995884773663
my_regression 0.5059670781893004
two_moons 0.6314403292181069
